In [ ]:
%%capture
!pip install baybe
import pandas as pd
import numpy as np
import datetime
from baybe.targets import NumericalTarget
from baybe.objectives.single import SingleTargetObjective
from baybe.constraints import ContinuousLinearInequalityConstraint

In [ ]:
parameters_data = [
    ("potential1",-2000,2000,1),
    ("hold1",0,1800,1),
    ("potential2",-2000,2000,1),
    ("hold2",0,300,1),
    ("potential3",-2000,2000,1),
    ("sweep_speed",10,10000,1),
    ("cycles",0,100,1),
]
constraints = None

from baybe.parameters import NumericalContinuousParameter
parameters = [
    NumericalContinuousParameter(
        name=p[0],
         bounds=(0, 1),      
    ) for p in parameters_data
]

In [ ]:
from baybe.searchspace import SearchSpace
from baybe.recommenders import (
    BotorchRecommender,
    RandomRecommender,
    TwoPhaseMetaRecommender,
)

recommender = TwoPhaseMetaRecommender(
    initial_recommender=RandomRecommender(),  # farthest point sampling
    recommender=BotorchRecommender(sequential_continuous=True),  # Bayesian model-based optimization
)

searchspace = SearchSpace.from_product(parameters,constraints=constraints)

In [ ]:
def scale_measurements_down(df, parameter_info):
    for p in parameter_info:
        values = (df[p[0]]-p[1])/(p[2]-p[1])
        df[p[0]] = values
    return df

def scale_parameters_up(df, parameter_info):
    for p in parameter_info:
        values = (p[2]-p[1])*df[p[0]] + p[1]
        df[p[0]] = round(values / p[3]) * p[3]
    return df

In [ ]:
from baybe import Campaign

target = NumericalTarget(name="Activity", mode="MAX")
objective = SingleTargetObjective(target)
columns = [p[0] for p in parameters_data]
campaign = Campaign(searchspace, objective, recommender)
measurements = pd.read_csv('baybe_csv/wateroxidation_ni.csv')

measurements = scale_measurements_down(measurements, parameters_data)
campaign.add_measurements(measurements)
df_next = campaign.recommend(batch_size=2)

print(df_next)
df_next = scale_parameters_up(df_next, parameters_data)

df_next


In [ ]:
measurements

In [ ]:
# make table as expected by student
interval1 = df_next['hold1'] + np.abs(df_next['potential1'] - df_next['potential2'])/df_next['sweep_speed']
interval2 = np.abs(df_next['potential2'] - df_next['potential3'])/df_next['sweep_speed'] + df_next['hold2']
df_next['duration'] = interval1 + df_next['cycles']*interval2
rename_columns = {
    'potential1': 'potential1 (mV vs RHE)',
    'hold1': 'hold1 (s)',
    'potential2': 'potential2 (mV vs RHE)',
    'hold2': 'hold2 (s)',
    'potential3': 'potential3 (mV vs RHE)',
    'sweep_speed': 'sweep speed (mV/s)',
    'cycles': 'cycle (P2-P3)',
    'duration': 'duration (s)',
}
df_next = df_next.rename(columns=rename_columns)

date_now = datetime.datetime.now()
filename = 'baybe_csv/newparameters_ni_' + date_now.strftime("%Y%m%d") + '.csv'
df_next.to_csv(filename)

df_next